In [1]:
import pandas as pd
import os
import random
import time
from albumentations import *
from albumentations.pytorch import ToTensor
import cv2
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
from torch.optim import Adam, SGD
from torch.nn import functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset, sampler
from tqdm import tqdm
import json
import matplotlib
import sys
import timm
import albumentations as A
from mmdet.apis import inference_detector, init_detector
from albumentations.pytorch import ToTensorV2
sys.path.append('./ensemble_boxes-1.0.4')
from ensemble_boxes import *
matplotlib.use('TkAgg')
from matplotlib import pyplot as plt
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
def run_nms(bboxes, confs,classs, image_size, iou_thr=0.50, skip_box_thr=0.0001, weights=None):
    boxes =  [bbox/(image_size-1) for bbox in bboxes]
    scores = [conf for conf in confs]    
    labels = [np.ones(conf.shape[0]) for conf in confs]
    boxes, scores, labels = nms(boxes, scores, labels, weights=weights, iou_thr=iou_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels

def run_nmw(bboxes, confs,classs, image_size, iou_thr=0.50, skip_box_thr=0.0001, weights=None):
    boxes =  [bbox/(image_size-1) for bbox in bboxes]
    scores = [conf for conf in confs]    
    labels = [np.ones(conf.shape[0]) for conf in confs]
    boxes, scores, labels = non_maximum_weighted(boxes, scores, labels, weights=weights, iou_thr=iou_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels


def run_soft_nms(bboxes, confs,classs, image_size, iou_thr=0.50, skip_box_thr=0.0001,sigma=0.1, weights=None):
    boxes =  [bbox/(image_size-1) for bbox in bboxes]
    scores = [conf for conf in confs]    
    labels = [np.ones(conf.shape[0]) for conf in confs]
    boxes, scores, labels = soft_nms(boxes, scores, labels, weights=weights, iou_thr=iou_thr, sigma=sigma, thresh=skip_box_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels

def run_wbf(bboxes, confs,classs, iou_thr=0.50, skip_box_thr=0.5, weights=None):
    boxes =  [bbox for bbox in bboxes]
    scores = [conf for conf in confs]    
    labels = [cls for cls in classs]
    #print(labels)
    #labels = convert_same(labels)
    #print(labels)
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    #boxes = boxes*(image_size-1)
    return boxes, scores, labels
def load_model(ckpt_path, conf=0.25, iou=0.50):
    model = torch.hub.load('/home/liuyun/ali/vma/hist_log/focal_m_1280',
                           'custom',
                           path=ckpt_path,
                           source='local',
                           force_reload=True)  # local repo
    model.conf = conf  # NMS confidence threshold
    model.iou  = iou  # NMS IoU threshold
    model.classes = None   # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs
    model.multi_label = False  # NMS multiple labels per box
    #model.agnostic = True
    model.max_det = 1000  # maximum number of detections per image
    return model

class CustomModel(nn.Module):
    def __init__(self, pretrained=False):
        super().__init__()
        self.model = timm.create_model('tf_efficientnetv2_s', pretrained=pretrained, in_chans=3)
        #self.model.load_state_dict(torch.load('../data/tf_efficientnetv2_s-eb54923e.pth'))
        self.n_features = self.model.classifier.in_features
        self.model.global_pool = nn.Identity()
        self.model.classifier = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Linear(self.n_features, 8)

    def forward(self, x, labels=None):
        bs = x.size(0)
        features = self.model(x)
        pool_feature = self.pooling(features).view(bs, -1)
        output = self.classifier(pool_feature)
        return output
def clasifier_part(input_img,det_result,cls_models):
    transforms = A.Compose([
            A.Resize(128, 128),
            ToTensorV2(),
        ])
    img = input_img[int(det_result[1]):int(det_result[3]),int(det_result[0]):int(det_result[2]),:]
    #cv2.imwrite('testpart.png',img)
    img = transforms(image=img)['image'] / 255.0
    img = img.to(device)
    final_out = None
    with torch.no_grad():
        for model in cls_models:
            out = F.softmax(model(img.unsqueeze(0)),dim=1)
            if final_out is not None:
                final_out += out
            else:
                final_out = out
    final_out /= len(cls_models)
    final_pred = torch.argmax(final_out)
    return int(final_pred)
def predict(model, img, size=1280, augment=False):
    height, width = img.shape[:2]
    #cv2.imwrite('./test.png',img)
    #img = np.ascontiguousarray(img)
    with torch.no_grad():
        results = model(img, size=size, augment=augment)  # custom inference size
    preds = results.pandas().xyxyn[0]
    #bboxes = preds[['xcenter', 'ycenter', 'width', 'height']].values
    bboxes = preds[['xmin', 'ymin', 'xmax', 'ymax']].values
    #x1 = int(bboxes[0][0] - bboxes[0][2] // 2)
    #y1 = int(bboxes[0][1] - bboxes[0][3] // 2)
    #x2 = int(bboxes[0][0] + bboxes[0][2] // 2)
    #y2 = int(bboxes[0][1] + bboxes[0][3] // 2)
    #cv2.rectangle(img,(x1,y1),(x2,y2),(255, 0, 0), 1)
    #cv2.imwrite('./test_box.png',img)
    if len(bboxes):
        bboxes = bboxes#voc2coco(bboxes, height, width).astype(int)
        confs = preds.confidence.values
        classes = preds['class'].values
        return bboxes.tolist(), list(confs),list(classes)
    else:
        return [], [],[]


def predict_mmdet(model,path):
    img = cv2.imread(path)
    w,h = img.shape[:2]
    result = inference_detector(model, path)
    bbox_result = result
    bboxes = np.vstack(bbox_result)
    confis = [bboxes[i][-1] for i in range(0, len(bboxes))]
    classes = [
        np.full(bbox.shape[0], i, dtype=np.int32)
        for i, bbox in enumerate(bbox_result)
    ]
    classes = np.concatenate(classes)
    only_boxes = bboxes[:,:4]
   
    only_boxes[:,0] /= h
    only_boxes[:,1] /= w
    only_boxes[:,2] /= h
    only_boxes[:,3] /= w
    
    if len(confis) == 0:
        return [],[],[]
    else:
        return only_boxes.tolist(),confis,classes.tolist()
def convert_same(classes):
    vote_num =  np.zeros((5))
    lenth = 0
    total_classes = []
    for m_classes in classes:
        lenth += len(m_classes)
        total_classes += m_classes
    if 5 in total_classes or 6 in total_classes or 7 in total_classes or lenth <= 10:
        return classes
    else:
        for cls in total_classes:
            vote_num[cls] += 1
    max_idx = int(np.where(vote_num == np.max(vote_num))[0][0])
    for i,m_cls in enumerate(classes):
        for j,cur_cls in enumerate(m_cls):
            classes[i][j] = max_idx
    
    return classes


def main():
    #cls model
    model_cls_0 = CustomModel()
    model_cls_0.load_state_dict(torch.load(f'/home/liuyun/ali/vma/hist_weights/log1013/tf_efficientnetv2_s_fold0_best_score.pth',map_location=torch.device('cpu'))['model'])
    model_cls_0.to(device)
    model_cls_0.eval()
    model_cls_1 = CustomModel()
    model_cls_1.load_state_dict(torch.load(f'/home/liuyun/ali/vma/hist_weights/log1013/tf_efficientnetv2_s_fold1_best_score.pth',map_location=torch.device('cpu'))['model'])
    model_cls_1.to(device)
    model_cls_1.eval()
    model_cls_2 = CustomModel()
    model_cls_2.load_state_dict(torch.load(f'/home/liuyun/ali/vma/hist_weights/log1013/tf_efficientnetv2_s_fold2_best_score.pth',map_location=torch.device('cpu'))['model'])
    model_cls_2.to(device)
    model_cls_2.eval()
    model_cls_3 = CustomModel()
    model_cls_3.load_state_dict(torch.load(f'/home/liuyun/ali/vma/hist_weights/log1013/tf_efficientnetv2_s_fold3_best_score.pth',map_location=torch.device('cpu'))['model'])
    model_cls_3.to(device)
    model_cls_3.eval()
    model_cls_4 = CustomModel()
    model_cls_4.load_state_dict(torch.load(f'/home/liuyun/ali/vma/hist_weights/log1013/tf_efficientnetv2_s_fold4_best_score.pth',map_location=torch.device('cpu'))['model'])
    model_cls_4.to(device)
    model_cls_4.eval()
    
    model_cls = [model_cls_0,model_cls_1,model_cls_2,model_cls_3,model_cls_4]
    labels = []
    #yolo model
    CKPT_PATH_0 = '/home/liuyun/ali/vma/hist_log/focal_m_1280/runs/log1009/baseline_fold0/weights/best.pt'
    CKPT_PATH_1 = '/home/liuyun/ali/vma/hist_log/focal_m_1280/runs/log1009/baseline_fold1/weights/best.pt'
    CKPT_PATH_2 = '/home/liuyun/ali/vma/hist_log/focal_m_1280/runs/log1009/baseline_fold2/weights/best.pt'
    CKPT_PATH_3 = '/home/liuyun/ali/vma/hist_log/focal_m_1280/runs/log1009/baseline_fold3/weights/best.pt'
    CKPT_PATH_4 = '/home/liuyun/ali/vma/hist_log/focal_m_1280/runs/log1009/baseline_fold4/weights/best.pt'
    Test_files = os.listdir('../data/test_images/images')
    IMG_SIZE = 1280
    AUGMENT = True
    #model_0 = load_model(CKPT_PATH_0, conf=0.27, iou=0.5)
    #model_1 = load_model(CKPT_PATH_1, conf=0.61, iou=0.5)
    #model_2 = load_model(CKPT_PATH_2, conf=0.4, iou=0.5)
    #model_3 = load_model(CKPT_PATH_3, conf=0.55, iou=0.5)
    #model_4 = load_model(CKPT_PATH_4, conf=0.6, iou=0.5)
    #mmdet cascade rcnn model
    # config文件
    config_file = '/home/liuyun/ali/vma/mmdetection-master/save_models/heavy_alb_enhance_mosaic/fold0/cascade_rcnn_coco_alb_mix.py'
    checkpoint_file = '/home/liuyun/ali/vma/mmdetection-master/save_models/heavy_alb_enhance_mosaic/fold0/best_bbox_mAP_epoch_41.pth'#'./checkpoint/best_bbox_mAP_epoch_146.pth'
    model_cascade_0 = init_detector(config_file, checkpoint_file, device='cuda:0')
    
    
    results = []
    for idx, name in tqdm(enumerate(Test_files),total=len(Test_files)):
        cur_name = f'test{idx}'#name.split('.')[0]
        file_path = f'../data/test_images/images/test{idx}.png'
        #yolo inference
        img = cv2.imread(file_path)[...,::-1]
        height_ori, width_ori = img.shape[:2]
        img = np.ascontiguousarray(img)
        #bboxes_0, confis_0,classes_0 = predict(model_0, img, size=IMG_SIZE, augment=AUGMENT)
        #bboxes_1, confis_1,classes_1 = predict(model_1, img, size=IMG_SIZE, augment=AUGMENT)
        #bboxes_2, confis_2,classes_2 = predict(model_2, img, size=IMG_SIZE, augment=AUGMENT)
        #bboxes_3, confis_3,classes_3 = predict(model_3, img, size=IMG_SIZE, augment=AUGMENT)
        #bboxes_4, confis_4,classes_4 = predict(model_4, img, size=IMG_SIZE, augment=AUGMENT)
        
        
        #mmdet inference
        #-------------------------
        bboxes_5, confis_5,classes_5 = predict_mmdet(model_cascade_0,file_path)
        
        
        #-------------------------
        bboxes = [bboxes_5]#[bboxes_0,bboxes_1,bboxes_2,bboxes_3,bboxes_4]
        confis = [confis_5]#[confis_0,confis_1,confis_2,confis_3,confis_4]
        classes = [classes_5]#[classes_0,classes_1,classes_2,classes_3,classes_4]
        bboxes,confis,classes = run_wbf(bboxes, confis, classes, iou_thr=0.5,weights=[1,1,1,1,1])
        
        if len(bboxes) != 0:
            for i in range(len(bboxes)):
                
                cur_box = bboxes[i]
                cur_box[0] *= float(width_ori)
                cur_box[1] *= float(height_ori)
                cur_box[2] *= float(width_ori)
                cur_box[3] *= float(height_ori)
                #cls part-------------
                #cls_pred = clasifier_part(img,cur_box,model_cls)
                xmin = cur_box[0]
                ymin = cur_box[1] 
                width= cur_box[2] - cur_box[0]
                height= cur_box[3] - cur_box[1]
                cur_box_xywh = [int(xmin),int(ymin),int(width),int(height)]
                cur_confi = float('%.4f' % confis[i])
                cur_cls = classes[i]#cls_pred#
                
                cv2.rectangle(img, (int(cur_box[0]), int(cur_box[1])), (int(cur_box[2]), int(cur_box[3])), (255, 0, 0), 1)
                cv2.putText(img,str(cur_cls),(int(cur_box[0]), int(cur_box[1])),cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)
                results.append({"image_id":cur_name,"category_id":int(cur_cls),"bbox":cur_box_xywh,"score":cur_confi})
        cv2.imwrite(f'../data/plot_result/{cur_name}.png',img)
        
    with open('sub.json', 'w') as outfile:
        json.dump(results, outfile)
if __name__ == '__main__':
    main()

2022-10-15 16:08:44,496 - mmcls - INFO - backbone out_indices: (1, 2, 3, 4)
2022-10-15 16:08:44,496 - mmcls - INFO - backbone out_channels: [48, 64, 160, 256]
2022-10-15 16:08:44,497 - mmcls - INFO - backbone out_strides: [4, 8, 16, 32]


load checkpoint from local path: /home/liuyun/ali/vma/mmdetection-master/save_models/heavy_alb_enhance_mosaic/fold0/best_bbox_mAP_epoch_41.pth


100%|██████████| 430/430 [02:36<00:00,  2.75it/s]
